In [33]:
import pickle
from bareunpy import Tagger

import time

In [34]:
# 바른 형태소 분석기 정의
API_KEY = "--"
baruen_tagger = Tagger(API_KEY, 'localhost')

In [35]:
def tokenize(s):
    return [token for token in baruen_tagger.morphs(s)]

In [36]:
# 토큰화&자모 분리 시작
count = 0

# 토큰화 결과를 담을 리스트
tokenized_data = []

for sample in data['Content'].to_list():
    start_time = time.time()
    tokenzied_sample = tokenize(sample)
    tokenized_data.append(tokenzied_sample)
    count += 1

    # 경과시간 출력
    if count % 100 == 0:
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'{count} sample processed. Elapsed time : {round(elapsed_time // 60, 0)}m {round(elapsed_time % 60, 0)}s')

100 sample processed. Elapsed time : 0.0m 0.0s
200 sample processed. Elapsed time : 0.0m 0.0s
300 sample processed. Elapsed time : 0.0m 0.0s
400 sample processed. Elapsed time : 0.0m 0.0s
500 sample processed. Elapsed time : 0.0m 0.0s
600 sample processed. Elapsed time : 0.0m 0.0s
700 sample processed. Elapsed time : 0.0m 0.0s
800 sample processed. Elapsed time : 0.0m 0.0s
900 sample processed. Elapsed time : 0.0m 0.0s
1000 sample processed. Elapsed time : 0.0m 0.0s
1100 sample processed. Elapsed time : 0.0m 0.0s
1200 sample processed. Elapsed time : 0.0m 0.0s
1300 sample processed. Elapsed time : 0.0m 0.0s
1400 sample processed. Elapsed time : 0.0m 0.0s
1500 sample processed. Elapsed time : 0.0m 0.0s
1600 sample processed. Elapsed time : 0.0m 0.0s
1700 sample processed. Elapsed time : 0.0m 0.0s
1800 sample processed. Elapsed time : 0.0m 0.0s
1900 sample processed. Elapsed time : 0.0m 0.0s
2000 sample processed. Elapsed time : 0.0m 0.0s
2100 sample processed. Elapsed time : 0.0m 0.0s
2

In [37]:
with open('Part2_Randomized_Data_to_Token.pkl',"wb") as f:
    pickle.dump(tokenized_data, f)

In [38]:
with open("Part2_Randomized_Data_to_Token.pkl","rb") as f:
    tokenized_data = pickle.load(f)

In [40]:
from gensim.models import Word2Vec

# 모델 구성하기(fasttext와 다르게 skip-gram)
model = Word2Vec(vector_size=128, window=5, min_count=3,
                 sg=1, negative=5)

In [41]:
%%time

# 단어집합 만들기
model.build_vocab(tokenized_data, progress_per=10000)

CPU times: total: 6.94 s
Wall time: 7.29 s


In [42]:
# 초기 임베딩 행렬의 크기
model.wv.vectors.shape

(99129, 128)

In [43]:
%%time

# 임베딩 학습
model.train(tokenized_data, total_examples=model.corpus_count, epochs=10)

CPU times: total: 29min 33s
Wall time: 11min 6s


(187245146, 257378100)

In [44]:
word_vectors = model.wv

In [45]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["영업"])

[('이익률', 0.7283915877342224),
 ('매출액', 0.7119492888450623),
 ('매출', 0.708001434803009),
 ('순이익', 0.700469970703125),
 ('이익', 0.6981784701347351),
 ('타이어보강재', 0.6545750498771667),
 ('분기', 0.6368043422698975),
 ('5118', 0.6311440467834473),
 ('순손실', 0.6281888484954834),
 ('감가상각비', 0.6266810894012451)]

In [46]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["제품"])

[('수퍼슬림', 0.7119824886322021),
 ('품목', 0.7087818384170532),
 ('제품군', 0.7074756622314453),
 ('펄프몰드', 0.7026435136795044),
 ('라펜', 0.7013533711433411),
 ('멀티쿠커', 0.6986954212188721),
 ('분해기', 0.6975739002227783),
 ('제조업자', 0.6951971054077148),
 ('DDIC', 0.6930301785469055),
 ('벌크', 0.6904259324073792)]

In [47]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["경제"])

[('거시', 0.6887995004653931),
 ('기획원', 0.6616912484169006),
 ('안보', 0.6521635055541992),
 ('현오석', 0.6508649587631226),
 ('추광호', 0.647553026676178),
 ('성장률', 0.635644257068634),
 ('사회', 0.6351566314697266),
 ('폭망', 0.6349054574966431),
 ('일자리기획관', 0.6342824697494507),
 ('대외', 0.626801609992981)]

In [48]:
# 양의 상관관계를 갖는 단어 출력 -> OOV는 처리하지 못함
word_vectors.most_similar(positive=["뭐야"])

KeyError: "Key '뭐야' not present in vocabulary"

In [49]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["돈"])

[('벌', 0.7716547250747681),
 ('큰돈', 0.7414222359657288),
 ('융통하', 0.7124620079994202),
 ('차입하', 0.7070488929748535),
 ('푼', 0.7044523358345032),
 ('빌려주', 0.695864737033844),
 ('나랏돈', 0.6794853210449219),
 ('호주머니', 0.6765522956848145),
 ('탕진하', 0.6724694967269897),
 ('종잣돈', 0.6709563136100769)]

In [50]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["삼성"])

[('삼성전자', 0.7720504999160767),
 ('SDS', 0.728722095489502),
 ('SDI', 0.7279341220855713),
 ('LG', 0.6680448651313782),
 ('삼성전기', 0.6633411049842834),
 ('이재용', 0.6530425548553467),
 ('고동진', 0.6511996388435364),
 ('전영현', 0.6421107053756714),
 ('삼성그룹', 0.6359306573867798),
 ('이노텍', 0.6312949657440186)]

In [51]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["LG"])

[('LG전자', 0.7927642464637756),
 ('이노텍', 0.7612348794937134),
 ('CNS', 0.741729199886322),
 ('듀얼업', 0.7292957305908203),
 ('삼성전자', 0.7215280532836914),
 ('에어로타워', 0.7136680483818054),
 ('QNED', 0.6971585750579834),
 ('가이드봇', 0.6965489387512207),
 ('프로빔', 0.6921705603599548),
 ('MiniLED', 0.6918479204177856)]

In [52]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["취준생"])

[('남군', 0.7065442204475403),
 ('면접관', 0.6626892685890198),
 ('고군분투기', 0.6466269493103027),
 ('상담자', 0.6453847289085388),
 ('문제아', 0.6440674662590027),
 ('불합격하', 0.6403819918632507),
 ('주환', 0.6358957886695862),
 ('리사코', 0.6355201005935669),
 ('하숙', 0.6328415274620056),
 ('가사일', 0.6246992349624634)]

In [53]:
# 양의 상관관계를 갖는 단어 출력 -> OOV
word_vectors.most_similar(positive=["ㄹㅇ"])

KeyError: "Key 'ㄹㅇ' not present in vocabulary"

In [54]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["사과"])

[('사죄', 0.7462431192398071),
 ('사과하', 0.6862327456474304),
 ('일언반구', 0.6561250686645508),
 ('돌잡이', 0.6408822536468506),
 ('송구하', 0.6359294056892395),
 ('반성', 0.6317437887191772),
 ('정중히', 0.6313472390174866),
 ('사죄하', 0.6261341571807861),
 ('사과문', 0.6226144433021545),
 ('조의', 0.6162630319595337)]

In [55]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["코딩"])

[('프로그래밍', 0.7958728671073914),
 ('학습', 0.7582099437713623),
 ('비브라스', 0.72721266746521),
 ('SW', 0.7187265157699585),
 ('소프트웨어', 0.698569118976593),
 ('스마트올', 0.6966381072998047),
 ('지도안', 0.6960616111755371),
 ('커리큘럼', 0.6907548904418945),
 ('클래스팅', 0.6904010772705078),
 ('코듀에이스', 0.6901008486747742)]

In [56]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["파이썬"])

[('Java', 0.7788122892379761),
 ('비브라스', 0.7777286767959595),
 ('캐치잇잉글리시', 0.7768628597259521),
 ('큐티포파이썬', 0.7767699360847473),
 ('라즈베리파이', 0.7704955339431763),
 ('텐서플로', 0.7688829302787781),
 ('커리어셀', 0.7670580744743347),
 ('아키텍쳐', 0.7662830352783203),
 ('엘리먼트', 0.7661173343658447),
 ('래블업', 0.7649107575416565)]

In [57]:
# 양의 상관관계를 갖는 단어 출력
word_vectors.most_similar(positive=["컴퓨터"])

[('아두이노', 0.6784263253211975),
 ('HMD', 0.6575337648391724),
 ('현미경', 0.6518418192863464),
 ('자이', 0.6494518518447876),
 ('CNC', 0.6448395252227783),
 ('ECU', 0.6413640975952148),
 ('노트북', 0.6381550431251526),
 ('소프트웨어', 0.6313837170600891),
 ('PC', 0.6289246082305908),
 ('CAD', 0.6277445554733276)]

In [58]:
# 단어 사이의 관계
word_vectors.most_similar(positive=["미국", "기업"])

[('유사코그룹', 0.709170401096344),
 ('SAMR', 0.6855641007423401),
 ('글로벌', 0.6808845400810242),
 ('파이티움', 0.6770892143249512),
 ('무역국', 0.6724931001663208),
 ('조달처', 0.6684592962265015),
 ('중국', 0.6675516366958618),
 ('센스타임', 0.6660979986190796),
 ('중촨웨중', 0.6637479662895203),
 ('SMIC', 0.6623342037200928)]

In [59]:
# 단어 사이의 관계
word_vectors.most_similar(positive=["한국", "기업"])

[('중소기업', 0.7042385935783386),
 ('국제무역통상연구원', 0.6872962117195129),
 ('중소기업부', 0.6816702485084534),
 ('전자업체', 0.6691113710403442),
 ('유사코그룹', 0.6669377684593201),
 ('대·중견', 0.6665489673614502),
 ('기업인', 0.6639979481697083),
 ('트루윈', 0.6620478630065918),
 ('TYM', 0.6594043970108032),
 ('기업데이터', 0.6573240160942078)]

In [60]:
# 단어 사이의 관계
word_vectors.most_similar(positive=["사람", "나이"], negative=['노인'])

[('예순', 0.6061387062072754),
 ('저분', 0.6018008589744568),
 ('탄로', 0.6014621257781982),
 ('꾸역꾸역', 0.5995429754257202),
 ('치욕스럽', 0.5875227451324463),
 ('술친구', 0.5870833992958069),
 ('서른셋', 0.5835957527160645),
 ('존경스럽', 0.5826271772384644),
 ('새언니', 0.5811287760734558),
 ('반려자', 0.579098105430603)]

In [64]:
# 5만개로 학습한 Word2Vec 모델 저장
model.save("Part2_Word2Vec.model")